In [1]:
#Librerias
import pandas as pd
import numpy as np
import os

In [2]:
path = 'Data/'
sports = pd.read_excel(path+'sports.xlsx')

In [3]:
sports.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament
0,2.20,2000-08-19,H,1,Football,E0
1,1.60,2000-08-19,H,1,Football,E0
2,2.20,2000-08-19,H,0,Football,E0
3,1.80,2000-08-19,H,0,Football,E0
4,1.55,2000-08-19,H,1,Football,E0


In [4]:
#Creamos una función que convierta la cuota en una probabilidad, para así hacer un análisis más sencillo.
#Esta probabilidad será la que tiene de ganar el favorito

def prob(a):
    return (1/a)*100

In [5]:
sports['Probabilidad'] = sports['CuotaFavorito'].apply(prob)

In [6]:
sports.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament,Probabilidad
0,2.20,2000-08-19,H,1,Football,E0,45.454545
1,1.60,2000-08-19,H,1,Football,E0,62.500000
2,2.20,2000-08-19,H,0,Football,E0,45.454545
3,1.80,2000-08-19,H,0,Football,E0,55.555556
4,1.55,2000-08-19,H,1,Football,E0,64.516129


In [7]:
sports.dtypes

CuotaFavorito           float64
Date             datetime64[ns]
Favorito                 object
GanaFavorito              int64
Sport                    object
Tournament               object
Probabilidad            float64
dtype: object

## El primer análisis que haremos será apostar 10€ siempre al favorito.

In [8]:
sports['Hucha10'] = np.where(sports['GanaFavorito'] == 1, sports['CuotaFavorito']*10-10, -10)

In [9]:
sports.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament,Probabilidad,Hucha10
0,2.20,2000-08-19,H,1,Football,E0,45.454545,12.0
1,1.60,2000-08-19,H,1,Football,E0,62.500000,6.0
2,2.20,2000-08-19,H,0,Football,E0,45.454545,-10.0
3,1.80,2000-08-19,H,0,Football,E0,55.555556,-10.0
4,1.55,2000-08-19,H,1,Football,E0,64.516129,5.5


In [10]:
sports['Hucha10'].sum()

-163289.0

In [11]:
#La segunda Hucha la calcularemos multiplicando su cuota por la probabilidad de ganar
sports['HuchaB'] = np.where(sports['GanaFavorito'] == 1, (sports['CuotaFavorito']*sports['Probabilidad']/10)-(sports['Probabilidad']/10), -sports['Probabilidad']/10)

In [12]:
sports['HuchaB'].sum()

-92195.14371700516

La hucha sale negativa, entonces tendremos que buscar un indicador que nos diga cuanto apostar a cada cuota.

In [13]:
#Cogemos solamente los favoritos que ganan

In [14]:
favoritos = sports[sports['GanaFavorito'] == 1]
favoritos.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament,Probabilidad,Hucha10,HuchaB
0,2.20,2000-08-19,H,1,Football,E0,45.454545,12.0,5.454545
1,1.60,2000-08-19,H,1,Football,E0,62.500000,6.0,3.750000
4,1.55,2000-08-19,H,1,Football,E0,64.516129,5.5,3.548387
6,1.25,2000-08-19,H,1,Football,E0,80.000000,2.5,2.000000
8,1.50,2000-08-19,H,1,Football,E0,66.666667,5.0,3.333333


In [15]:
#Agrupado por cuotas

favoritoAgrupados = favoritos.groupby('CuotaFavorito',as_index=False)['GanaFavorito'].sum()
favoritoAgrupados.head()

,CuotaFavorito,GanaFavorito
0,1.000,96
1,1.001,1
2,1.002,57
3,1.003,1
4,1.005,103


In [16]:
sportsTotal = sports.groupby('CuotaFavorito',as_index=False)['Favorito'].count()
sportsTotal.head()

,CuotaFavorito,Favorito
0,1.000,96
1,1.001,1
2,1.002,60
3,1.003,1
4,1.005,103


In [17]:
#Juntamos las tablas para poder obtener el porcentaje de favoritos que ganan teniendo una cuota concreta
sportsTotalFavoritos = favoritoAgrupados.merge(sportsTotal, how='left')
sportsTotalFavoritos['Porcentaje'] = sportsTotalFavoritos['GanaFavorito']/sportsTotalFavoritos['Favorito']
sportsTotalFavoritos['Probabilidad'] = sportsTotalFavoritos['CuotaFavorito'].apply(prob)
sportsTotalFavoritos['Indicador'] = sportsTotalFavoritos['Porcentaje'] * sportsTotalFavoritos['Probabilidad']
sportsTotalFavoritos = sportsTotalFavoritos[['CuotaFavorito','Indicador']]
sportsTotalFavoritos.head()

,CuotaFavorito,Indicador
0,1.000,100.000000
1,1.001,99.900100
2,1.002,94.810379
3,1.003,99.700897
4,1.005,99.502488


In [18]:
sports = sports.merge(sportsTotalFavoritos, how = 'left')

In [19]:
sports.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament,Probabilidad,Hucha10,HuchaB,Indicador
0,2.20,2000-08-19,H,1,Football,E0,45.454545,12.0,5.454545,19.243025
1,1.60,2000-08-19,H,1,Football,E0,62.500000,6.0,3.750000,37.083788
2,2.20,2000-08-19,H,0,Football,E0,45.454545,-10.0,-4.545455,19.243025
3,1.80,2000-08-19,H,0,Football,E0,55.555556,-10.0,-5.555556,28.955330
4,1.55,2000-08-19,H,1,Football,E0,64.516129,5.5,3.548387,38.679065


In [20]:
sports['HuchaC'] = np.where(sports['GanaFavorito'] == 1, sports['CuotaFavorito']*sports['Indicador']/10, -sports['Indicador']/10)

In [21]:
sports.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament,Probabilidad,Hucha10,HuchaB,Indicador,HuchaC
0,2.20,2000-08-19,H,1,Football,E0,45.454545,12.0,5.454545,19.243025,4.233465
1,1.60,2000-08-19,H,1,Football,E0,62.500000,6.0,3.750000,37.083788,5.933406
2,2.20,2000-08-19,H,0,Football,E0,45.454545,-10.0,-4.545455,19.243025,-1.924302
3,1.80,2000-08-19,H,0,Football,E0,55.555556,-10.0,-5.555556,28.955330,-2.895533
4,1.55,2000-08-19,H,1,Football,E0,64.516129,5.5,3.548387,38.679065,5.995255


In [22]:
print(sports['Hucha10'].sum())
print(sports['HuchaB'].sum())
print(sports['HuchaC'].sum())

-163289.0
-92195.14371700516
521926.3007665614


In [23]:
#Gracias a las indicador hemos obtenido beneficio apostando al favorito.

Ahora vamos a calcular las tres huchas para Fútbol, Baloncesto y Tenis.

In [24]:
sports.groupby('Sport', as_index=False)[['Hucha10','HuchaB','HuchaC']].sum()

,Sport,Hucha10,HuchaB,HuchaC
0,Basketball,-4328.40,-3039.198699,40828.836709
1,Football,-99739.79,-50062.410331,219260.919411
2,Tennis,-59220.81,-39093.534687,261836.544646


Para finalizar vamos a analizar los 10 torneos de fútbol que son más rentables si apostamos al favorito.

In [33]:
footballTournament = sports[sports['Sport'] == 'Football'].groupby('Tournament', as_index=False)[['Hucha10','HuchaB','HuchaC']].sum()
footballTournament.sort_values(by='HuchaC', ascending=False).head(10)

,Tournament,Hucha10,HuchaB,HuchaC
9,E0,-2961.2,-1454.367782,12004.987533
35,SP1,-3516.6,-1804.004227,11938.415177
19,I1,-2196.9,-1096.944616,11165.160367
25,N1,-3287.7,-1775.200176,10887.508277
10,E1,-8819.5,-4367.152319,9922.657594
11,E2,-9223.6,-4648.419202,9793.945794
27,P1,-2556.1,-1248.543651,9162.418317
14,F1,-2575.3,-1404.078835,8963.830176
12,E3,-10294.0,-5165.155493,8899.938420
40,T1,-2613.0,-1421.206423,8832.606653


# Estadística descriptiva

In [40]:
datos = sports[['CuotaFavorito','Probabilidad','HuchaC','Sport']]

#### Datos estadísticos globales

In [44]:
datos.describe()

,CuotaFavorito,Probabilidad,HuchaC
count,245322.000000,245322.000000,245054.000000
mean,1.768716,60.004612,2.129842
std,0.422697,14.966664,4.718340
min,1.000000,21.978022,-9.705080
25%,1.430000,47.619048,-2.320227
50%,1.720000,58.139535,4.233465
75%,2.100000,69.930070,6.013782
max,4.550000,100.000000,10.000000


In [45]:
datos.mode()

,CuotaFavorito,Probabilidad,HuchaC,Sport
0,2.2,45.454545,-1.677671,Football


#### Datos estadísticos Fútbol

In [49]:
datos[datos['Sport'] == 'Football'].describe()

,CuotaFavorito,Probabilidad,HuchaC
count,160079.00000,160079.000000,160060.000000
mean,1.96442,52.943385,1.369867
std,0.36955,11.120552,4.005049
min,1.01000,21.978022,-8.839223
25%,1.70000,43.478261,-2.121569
50%,2.00000,50.000000,-0.983452
75%,2.30000,58.823529,4.962825
max,4.55000,99.009901,10.000000


In [50]:
datos[datos['Sport'] == 'Football'].mode()

,CuotaFavorito,Probabilidad,HuchaC,Sport
0,2.2,45.454545,-1.677671,Football


In [57]:
datos[datos['Sport'] == 'Football'].median()

CuotaFavorito     2.000000
Probabilidad     50.000000
HuchaC           -0.983452
dtype: float64

#### Datos estadísticos Baloncesto

In [51]:
datos[datos['Sport'] == 'Basketball'].describe()

,CuotaFavorito,Probabilidad,HuchaC
count,12059.000000,12059.000000,12059.000000
mean,1.451994,70.792934,3.385756
std,0.236604,11.947281,5.386958
min,1.000000,52.631579,-9.362175
25%,1.260000,60.975610,-3.342876
50%,1.440000,69.444444,5.983890
75%,1.640000,79.365079,7.549020
max,1.900000,100.000000,10.000000


In [52]:
datos[datos['Sport'] == 'Basketball'].mode()

,CuotaFavorito,Probabilidad,HuchaC,Sport
0,1.8,55.555556,5.211959,Basketball


In [56]:
datos[datos['Sport'] == 'Basketball'].median()

CuotaFavorito     1.440000
Probabilidad     69.444444
HuchaC            5.983890
dtype: float64

#### Datos estadísticos Tenis

In [53]:
datos[datos['Sport'] == 'Tennis'].describe()

,CuotaFavorito,Probabilidad,HuchaC
count,73184.000000,73184.000000,72935.000000
mean,1.392832,73.672324,3.589999
std,0.222253,11.908509,5.580432
min,1.000000,52.383447,-9.705080
25%,1.200000,63.694268,-3.342876
50%,1.390000,71.942446,6.271958
75%,1.570000,83.333333,7.746195
max,1.909000,100.000000,10.000000


In [54]:
datos[datos['Sport'] == 'Tennis'].mode()

,CuotaFavorito,Probabilidad,HuchaC,Sport
0,1.4,71.428571,6.790816,Tennis


In [55]:
datos[datos['Sport'] == 'Tennis'].median()

CuotaFavorito     1.390000
Probabilidad     71.942446
HuchaC            6.271958
dtype: float64